In [3]:
# only for gated models 
from huggingface_hub import login
login()

In [6]:
import sys
import os
import subprocess

print("Python executable:", sys.executable)
print("Python prefix:", sys.prefix)
print("Conda environment:", os.environ.get('CONDA_DEFAULT_ENV'))
print("Conda prefix:", os.environ.get('CONDA_PREFIX'))

# Check pip location
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Pip location:", result.stdout.strip())

# Check pip version
result = subprocess.run(['pip', '--version'], capture_output=True, text=True)
print("Pip version:", result.stdout.strip())

Python executable: /home/sagemaker-user/.conda/envs/inception/bin/python
Python prefix: /home/sagemaker-user/.conda/envs/inception
Conda environment: base
Conda prefix: /opt/conda
Pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip
Pip version: pip 25.2 from /home/sagemaker-user/.conda/envs/inception/lib/python3.9/site-packages/pip (python 3.9)


In [5]:
import os
import sys

# Add environment's bin directory to PATH
env_bin = os.path.dirname(sys.executable)
current_path = os.environ['PATH']
if env_bin not in current_path:
    os.environ['PATH'] = f"{env_bin}:{current_path}"

# Verify fix
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Fixed pip location:", result.stdout.strip())

Fixed pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip


In [7]:
pip install --upgrade transformers accelerate bitsandbytes torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,  # Threshold for outlier detection
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offloading if needed
)

# Load the model and tokenizer
model_path = "inceptionai/jais-13b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print(f"Model memory footprint: {model.get_memory_footprint() / 1024**3:.2f} GB")


tokenizer_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

configuration_jais.py:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

...

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model memory footprint: 12.69 GB


In [19]:
# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Prepare prompt
test_prompt = "What is artificial intelligence?"
inputs = tokenizer(test_prompt, return_tensors="pt")

# **Move inputs to the model's device**
input_ids = inputs.input_ids.to(device)

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Input:", test_prompt)
print("Output:", response)


Input: What is artificial intelligence?
Output: What is artificial intelligence?

Artificial intelligence (AI) is the science of making computers do things that would require intelligence if done by humans. AI is used in many fields including business, entertainment, and science.


In [20]:
# JAIS chat prompt
prompt = """### Instruction: Your name is Swapnil, and you are named after Jebel Jais.

### Input: [|Human|] Hello, can you introduce yourself?
### Response: [|AI|]"""

inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device)  # Move to GPU if available

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=100,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("### Response: [|AI|]")[-1].strip())


Hi, I'm Jais. I'm an AI assistant created by Inception and MBZUAI.
